In [1]:
import geopandas as gpd

import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import tensorflow as tf
import sys
import os
import glob

from functools import partial



code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRLinearModel
from make_datasets import make_data


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)
from zinf_gp.metrics import normcdf, fixed_top_X



from perturbations import perturbed
from bpr import bpr_variable_k_no_ties



2023-05-30 00:54:05.018132: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 00:54:05.020108: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 00:54:05.066062: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 00:54:05.066685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 00:54:07.141602: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
data_path='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/clean_annual_tract/'

In [8]:
epochs = 5000
seed = 360
time_window = 5
timesteps_per_year = 1
first_train_eval_year = 2013
last_train_eval_year = 2017
batch_dim_size = (last_train_eval_year - first_train_eval_year + 1)*timesteps_per_year
validation_year = 2018
first_test_year = 2019
last_test_year = 2020

tf.random.set_seed(seed)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']
y_idx_cols = [geography_col, timestep_col, outcome_col]
#features_only = ['lat', 'lon', timestep_col,
#                 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
#                 'svi_pctile',
#                 'neighbor_t', 'deaths']
features_only = ['deaths']

#data_gdf = gpd.read_file(data_path)

multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

train_x_BSF_flat, train_y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=1)

valid_x_BSF_flat, valid_y_BS = make_data(multiindexed_gdf, validation_year, validation_year,
                                         time_window, features_only, train_shape, pred_lag=1)

test_x_BSF_flat, test_y_BS = make_data(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=1)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)

2023-05-30 00:59:53.093195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-30 00:59:53.093283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: p1cmp078.pax.tufts.edu
2023-05-30 00:59:53.093309: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: p1cmp078.pax.tufts.edu
2023-05-30 00:59:53.093434: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 515.65.1
2023-05-30 00:59:53.093463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 515.65.1
2023-05-30 00:59:53.093469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 515.65.1


In [40]:
(0.5695636743215031+0.6083461538461538)/2

0.5889549140838284

In [37]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2017)&(multiindexed_gdf['year']>2010)].groupby(level='geoid')['deaths'].mean()
for quarter in [18]:
    this_2019 = multiindexed_gdf.loc[idx[:,quarter],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    bprs.append(fixed_top_X(this_2019,avg_2018,X=100)[-1])
np.mean(bprs)

0.5695636743215031

In [38]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2018)&(multiindexed_gdf['year']>2011)].groupby(level='geoid')['deaths'].mean()
for quarter in [19]:
    this_2019 = multiindexed_gdf.loc[idx[:,quarter],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    bprs.append(fixed_top_X(this_2019,avg_2018,X=100)[-1])
np.mean(bprs)

0.6083461538461538

In [39]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2019)&(multiindexed_gdf['year']>2012)].groupby(level='geoid')['deaths'].mean()
for quarter in [20]:
    this_2019 = multiindexed_gdf.loc[idx[:,quarter],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    bprs.append(fixed_top_X(this_2019,avg_2018,X=100)[-1])
np.mean(bprs)

0.564741482965932

In [14]:
bprs=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2020)&(multiindexed_gdf['year']>2015)].groupby(level='geoid')['deaths'].mean()
for quarter in [21]:
    this_2019 = multiindexed_gdf.loc[idx[:,quarter],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
    bprs.append(fixed_top_X(this_2019,avg_2018,X=100)[-1])
np.mean(bprs)

0.6046061185468451

In [23]:
this_2019.sort_values().iloc[-100:].sum()*0.02

10.46